In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
path = os.path.join(os.getcwd(), os.pardir)
sys.path.append(path)

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import norm, skew, lomax
from sklearn import metrics
import matplotlib.pyplot as plt
import pylab as py
from BAI import environment, solver, analysis

# Simulation Setup

In [4]:
saved_folder="K\=10_K\'\=2_eta\=1"
np.random.seed(1)
K = 10
K_H0 = 2
gamma = 0.01 * K
n_bandits = 10
n_parallels = 100000
n_batches = 20
batch_func = environment.func_constant_batch(500)
variance = [1 for i in range(K)]
lst_bandit_alter = []
lst_bandit_null = []
lst_thetas_alter = []
lst_thetas_null = []
for _ in range(n_bandits//2):
    # H1 is True - Low Signal
    thetas = np.sort(np.random.normal(0, 0.5, K))[::-1]
    reward_func_alter = environment.func_gaussian_reward(thetas, variance)
    bandit_alter = environment.Bandit(K=K, reward_func=reward_func_alter, batch_func=batch_func)
    lst_bandit_alter.append(bandit_alter)
    lst_thetas_alter.append(thetas)
    # H0 is True - Low Signal
    thetas = np.sort(np.random.normal(0, 0.5, K-K_H0+1))[::-1]
    best_mu = max(thetas)
    for _ in range(K_H0-1):
        thetas = np.insert(thetas, 1, best_mu)
    reward_func_null = environment.func_gaussian_reward(thetas, variance)
    bandit_null = environment.Bandit(K=K, reward_func=reward_func_null, batch_func=batch_func)
    lst_bandit_null.append(bandit_null)
    lst_thetas_null.append(thetas)

In [4]:
print("H0")
for each in lst_thetas_null:
    print(each)
print("H1")
for each in lst_thetas_alter:
    print(each)

H0
[ 0.73105397  0.73105397  0.56688472  0.02110687 -0.0862141  -0.1612086
 -0.19202718 -0.43892921 -0.54994563 -1.03007035]
[ 0.26517773  0.26517773  0.11720785 -0.0063323  -0.19837676 -0.33562307
 -0.34358635 -0.34583038 -0.42260282 -0.55865517]
[ 0.41949171  0.41949171  0.30860155  0.2933116   0.15008516  0.06007948
 -0.10444712 -0.17467136 -0.17612492 -0.5712591 ]
[ 1.0927877   1.0927877   0.75990841  0.43808446  0.15781747  0.08001853
 -0.25223293 -0.69824817 -0.7220569  -1.01110061]
[ 0.59945894  0.59945894  0.56474195  0.21174718  0.18878189  0.09257821
  0.06091064 -0.18764248 -0.3193652  -0.33533114]
H1
[ 0.87240588  0.81217268  0.43270381  0.15951955 -0.12468519 -0.26408588
 -0.30587821 -0.38060345 -0.53648431 -1.15076935]
[ 0.57236185  0.45079536  0.45042797  0.29140761  0.25124717 -0.06144511
 -0.13394404 -0.34186393 -0.46788472 -0.55030959]
[ 1.05012757  0.8462273   0.82990109  0.37102208  0.09545774  0.02540388
 -0.09591778 -0.31849782 -0.37357915 -0.44381448]
[ 0.6264340

In [5]:
eta = 1

# UNIF

In [6]:
G = 5000
beta = 0.5
tracking_history = []
for parallel_idx in range(n_parallels):
    bandit_choice = parallel_idx % n_bandits
    null_vs_alter = bandit_choice
    if bandit_choice >= n_bandits//2:
        bandit = lst_bandit_null[bandit_choice - n_bandits//2]
    elif bandit_choice < n_bandits//2:
        bandit = lst_bandit_alter[bandit_choice]
    trials = [0 for i in range(K)]
    rewards = [0 for i in range(K)]
    mus_nb = [0 for i in range(K)]
    mus_wb = [0 for i in range(K)]
    lambdas = [0 for i in range(K)]
    numerator = [0 for i in range(K)]
    denominator = [0 for i in range(K)]
    batch_count = [0 for i in range(K)]
    # cal sample variance
    SS = [0 for _ in range(K)]
    sample_variance_nb = [1 for _ in range(K)]
    sample_variance_wb = [1 for _ in range(K)]
    sampling_probs = [1 / K for i in range(K)]
    for batch_idx in range(n_batches):
        batch_feedback = bandit.batch_observe_fast(
            sampling_distrib = sampling_probs, 
            time_index=batch_idx
        )
        for arm_idx in range(K):
            batch_trials = batch_feedback['trials'][arm_idx]
            if batch_trials > 0:
                trials[arm_idx] += batch_trials
                rewards[arm_idx] += batch_feedback['rewards'][arm_idx]
                # update batch info
                batch_count[arm_idx] += 1
                denominator[arm_idx] += np.sqrt(batch_trials)
                numerator[arm_idx] += batch_feedback['rewards'][arm_idx] / np.sqrt(batch_trials)
                # initial prior by first batch or update posterior
                lambdas[arm_idx] =  denominator[arm_idx]**2 / batch_count[arm_idx]
                mus_wb[arm_idx] = numerator[arm_idx] / denominator[arm_idx]
                mus_nb[arm_idx] = rewards[arm_idx] / trials[arm_idx]
                # cal sample variance
                SS[arm_idx] += batch_feedback['squared_rewards'][arm_idx]
                sample_variance_nb[arm_idx] = SS[arm_idx] / trials[arm_idx] - mus_nb[arm_idx]**2
                sample_variance_wb[arm_idx] = SS[arm_idx] / trials[arm_idx] - mus_wb[arm_idx]**2
        # weighted batched
        mc_sampling = np.array([np.random.normal(mus_wb[i], np.sqrt(sample_variance_wb[i]/(eta * lambdas[i])), G) for i in range(K)])
        mc_optimal_arm = np.argmax(mc_sampling, axis=0)
        wb_posterior_delta = np.mean(np.max(mc_sampling, axis=0) - mc_sampling, axis=1)
        wb_optimal_probs = [round(np.sum(np.where(mc_optimal_arm == arm_i, 1, 0)) / G, 4) for arm_i in range(K)]
        # naive batched
        mc_sampling = np.array([np.random.normal(mus_nb[i], np.sqrt(sample_variance_nb[i]/(eta * trials[i])), G) for i in range(K)])
        mc_optimal_arm = np.argmax(mc_sampling, axis=0)
        nb_posterior_delta = np.mean(np.max(mc_sampling, axis=0) - mc_sampling, axis=1)
        nb_optimal_probs = [round(np.sum(np.where(mc_optimal_arm == arm_i, 1, 0)) / G, 4) for arm_i in range(K)]
        # tracking history and analysis
        for arm_idx in range(K):
            tracking_reccord = (
                parallel_idx,
                null_vs_alter,
                batch_idx,
                arm_idx,
                sampling_probs[arm_idx],
                mus_nb[arm_idx],
                trials[arm_idx],
                sample_variance_nb[arm_idx],
                nb_optimal_probs[arm_idx],
                nb_posterior_delta[arm_idx],
                mus_wb[arm_idx],
                lambdas[arm_idx],
                sample_variance_wb[arm_idx],
                wb_optimal_probs[arm_idx],
                wb_posterior_delta[arm_idx],
                trials[arm_idx] / sum(trials)
            )
            tracking_history.append(tracking_reccord)
        # change traffic
        # optimal_odds = [wb_optimal_probs[i] / (1 - wb_optimal_probs[i]) if wb_optimal_probs[i] < 1 else 99 for i in range(K)]
        # sampling_probs = [wb_optimal_probs[i] * (beta + (1 - beta) * (np.sum(optimal_odds) - optimal_odds[i])) for i in range(K)]
        # sampling_probs = [nb_optimal_probs[i] / np.sum(nb_optimal_probs) for i in range(K)]
        # sampling_probs = [(1 - gamma) * sampling_probs[i] + gamma * 1/K for i in range(K)]
# convert tracking_history to dataframe
df_UNIF = pd.DataFrame(tracking_history, columns=[
    'parallel_idx','null_vs_alter','batch_idx', 'arm_idx', 'sampling_prob', 
    'mu_nb', 'tau_nb', 'sample_variance_nb', 'optimal_prob_nb','posterior_delta_nb',
    'mu_wb', 'tau_wb', 'sample_variance_wb', 'optimal_prob_wb','posterior_delta_wb',
    'proportion_trials'
])
df_UNIF.to_csv(f"{saved_folder}/UNIF.csv", index=False)
del df_UNIF

# NB-TS

In [7]:
G = 5000
beta = 0.5
tracking_history = []
for parallel_idx in range(n_parallels):
    bandit_choice = parallel_idx % n_bandits
    null_vs_alter = bandit_choice
    if bandit_choice >= n_bandits//2:
        bandit = lst_bandit_null[bandit_choice - n_bandits//2]
    elif bandit_choice < n_bandits//2:
        bandit = lst_bandit_alter[bandit_choice]
    trials = [0 for i in range(K)]
    rewards = [0 for i in range(K)]
    mus_nb = [0 for i in range(K)]
    mus_wb = [0 for i in range(K)]
    lambdas = [0 for i in range(K)]
    numerator = [0 for i in range(K)]
    denominator = [0 for i in range(K)]
    batch_count = [0 for i in range(K)]
    # cal sample variance
    SS = [0 for _ in range(K)]
    sample_variance_nb = [1 for _ in range(K)]
    sample_variance_wb = [1 for _ in range(K)]
    sampling_probs = [1 / K for i in range(K)]
    for batch_idx in range(n_batches):
        batch_feedback = bandit.batch_observe_fast(
            sampling_distrib = sampling_probs, 
            time_index=batch_idx
        )
        for arm_idx in range(K):
            batch_trials = batch_feedback['trials'][arm_idx]
            if batch_trials > 0:
                trials[arm_idx] += batch_trials
                rewards[arm_idx] += batch_feedback['rewards'][arm_idx]
                # update batch info
                batch_count[arm_idx] += 1
                denominator[arm_idx] += np.sqrt(batch_trials)
                numerator[arm_idx] += batch_feedback['rewards'][arm_idx] / np.sqrt(batch_trials)
                # initial prior by first batch or update posterior
                lambdas[arm_idx] =  denominator[arm_idx]**2 / batch_count[arm_idx]
                mus_wb[arm_idx] = numerator[arm_idx] / denominator[arm_idx]
                mus_nb[arm_idx] = rewards[arm_idx] / trials[arm_idx]
                # cal sample variance
                SS[arm_idx] += batch_feedback['squared_rewards'][arm_idx]
                sample_variance_nb[arm_idx] = SS[arm_idx] / trials[arm_idx] - mus_nb[arm_idx]**2
                sample_variance_wb[arm_idx] = SS[arm_idx] / trials[arm_idx] - mus_wb[arm_idx]**2
        # weighted batched
        mc_sampling = np.array([np.random.normal(mus_wb[i], np.sqrt(sample_variance_wb[i]/(eta * lambdas[i])), G) for i in range(K)])
        mc_optimal_arm = np.argmax(mc_sampling, axis=0)
        wb_posterior_delta = np.mean(np.max(mc_sampling, axis=0) - mc_sampling, axis=1)
        wb_optimal_probs = [round(np.sum(np.where(mc_optimal_arm == arm_i, 1, 0)) / G, 4) for arm_i in range(K)]
        # naive batched
        mc_sampling = np.array([np.random.normal(mus_nb[i], np.sqrt(sample_variance_nb[i]/(eta * trials[i])), G) for i in range(K)])
        mc_optimal_arm = np.argmax(mc_sampling, axis=0)
        nb_posterior_delta = np.mean(np.max(mc_sampling, axis=0) - mc_sampling, axis=1)
        nb_optimal_probs = [round(np.sum(np.where(mc_optimal_arm == arm_i, 1, 0)) / G, 4) for arm_i in range(K)]
        # tracking history and analysis
        for arm_idx in range(K):
            tracking_reccord = (
                parallel_idx,
                null_vs_alter,
                batch_idx,
                arm_idx,
                sampling_probs[arm_idx],
                mus_nb[arm_idx],
                trials[arm_idx],
                sample_variance_nb[arm_idx],
                nb_optimal_probs[arm_idx],
                nb_posterior_delta[arm_idx],
                mus_wb[arm_idx],
                lambdas[arm_idx],
                sample_variance_wb[arm_idx],
                wb_optimal_probs[arm_idx],
                wb_posterior_delta[arm_idx],
                trials[arm_idx] / sum(trials)
            )
            tracking_history.append(tracking_reccord)
        # change traffic
        # optimal_odds = [wb_optimal_probs[i] / (1 - wb_optimal_probs[i]) if wb_optimal_probs[i] < 1 else 99 for i in range(K)]
        # sampling_probs = [wb_optimal_probs[i] * (beta + (1 - beta) * (np.sum(optimal_odds) - optimal_odds[i])) for i in range(K)]
        sampling_probs = [nb_optimal_probs[i] / np.sum(nb_optimal_probs) for i in range(K)]
        sampling_probs = [(1 - gamma) * sampling_probs[i] + gamma * 1/K for i in range(K)]
# convert tracking_history to dataframe
df_NBTS = pd.DataFrame(tracking_history, columns=[
    'parallel_idx','null_vs_alter','batch_idx', 'arm_idx', 'sampling_prob', 
    'mu_nb', 'tau_nb', 'sample_variance_nb', 'optimal_prob_nb','posterior_delta_nb',
    'mu_wb', 'tau_wb', 'sample_variance_wb', 'optimal_prob_wb','posterior_delta_wb',
    'proportion_trials'
])
df_NBTS.to_csv(f"{saved_folder}/NBTS.csv", index=False)
del df_NBTS

# WB-TS

In [8]:
G = 5000
beta = 0.5
tracking_history = []
for parallel_idx in range(n_parallels):
    bandit_choice = parallel_idx % n_bandits
    null_vs_alter = bandit_choice
    if bandit_choice >= n_bandits//2:
        bandit = lst_bandit_null[bandit_choice - n_bandits//2]
    elif bandit_choice < n_bandits//2:
        bandit = lst_bandit_alter[bandit_choice]
    trials = [0 for i in range(K)]
    rewards = [0 for i in range(K)]
    mus_nb = [0 for i in range(K)]
    mus_wb = [0 for i in range(K)]
    lambdas = [0 for i in range(K)]
    numerator = [0 for i in range(K)]
    denominator = [0 for i in range(K)]
    batch_count = [0 for i in range(K)]
    # cal sample variance
    SS = [0 for _ in range(K)]
    sample_variance_nb = [1 for _ in range(K)]
    sample_variance_wb = [1 for _ in range(K)]
    sampling_probs = [1 / K for i in range(K)]
    for batch_idx in range(n_batches):
        batch_feedback = bandit.batch_observe_fast(
            sampling_distrib = sampling_probs, 
            time_index=batch_idx
        )
        for arm_idx in range(K):
            batch_trials = batch_feedback['trials'][arm_idx]
            if batch_trials > 0:
                trials[arm_idx] += batch_trials
                rewards[arm_idx] += batch_feedback['rewards'][arm_idx]
                # update batch info
                batch_count[arm_idx] += 1
                denominator[arm_idx] += np.sqrt(batch_trials)
                numerator[arm_idx] += batch_feedback['rewards'][arm_idx] / np.sqrt(batch_trials)
                # initial prior by first batch or update posterior
                lambdas[arm_idx] =  denominator[arm_idx]**2 / batch_count[arm_idx]
                mus_wb[arm_idx] = numerator[arm_idx] / denominator[arm_idx]
                mus_nb[arm_idx] = rewards[arm_idx] / trials[arm_idx]
                # cal sample variance
                SS[arm_idx] += batch_feedback['squared_rewards'][arm_idx]
                sample_variance_nb[arm_idx] = SS[arm_idx] / trials[arm_idx] - mus_nb[arm_idx]**2
                sample_variance_wb[arm_idx] = SS[arm_idx] / trials[arm_idx] - mus_wb[arm_idx]**2
        # weighted batched
        mc_sampling = np.array([np.random.normal(mus_wb[i], np.sqrt(sample_variance_wb[i]/(eta * lambdas[i])), G) for i in range(K)])
        mc_optimal_arm = np.argmax(mc_sampling, axis=0)
        wb_posterior_delta = np.mean(np.max(mc_sampling, axis=0) - mc_sampling, axis=1)
        wb_optimal_probs = [round(np.sum(np.where(mc_optimal_arm == arm_i, 1, 0)) / G, 4) for arm_i in range(K)]
        # naive batched
        mc_sampling = np.array([np.random.normal(mus_nb[i], np.sqrt(sample_variance_nb[i]/(eta * trials[i])), G) for i in range(K)])
        mc_optimal_arm = np.argmax(mc_sampling, axis=0)
        nb_posterior_delta = np.mean(np.max(mc_sampling, axis=0) - mc_sampling, axis=1)
        nb_optimal_probs = [round(np.sum(np.where(mc_optimal_arm == arm_i, 1, 0)) / G, 4) for arm_i in range(K)]
        # tracking history and analysis
        for arm_idx in range(K):
            tracking_reccord = (
                parallel_idx,
                null_vs_alter,
                batch_idx,
                arm_idx,
                sampling_probs[arm_idx],
                mus_nb[arm_idx],
                trials[arm_idx],
                sample_variance_nb[arm_idx],
                nb_optimal_probs[arm_idx],
                nb_posterior_delta[arm_idx],
                mus_wb[arm_idx],
                lambdas[arm_idx],
                sample_variance_wb[arm_idx],
                wb_optimal_probs[arm_idx],
                wb_posterior_delta[arm_idx],
                trials[arm_idx] / sum(trials)
            )
            tracking_history.append(tracking_reccord)
        # change traffic
        # optimal_odds = [wb_optimal_probs[i] / (1 - wb_optimal_probs[i]) if wb_optimal_probs[i] < 1 else 99 for i in range(K)]
        # sampling_probs = [wb_optimal_probs[i] * (beta + (1 - beta) * (np.sum(optimal_odds) - optimal_odds[i])) for i in range(K)]
        sampling_probs = [wb_optimal_probs[i] / np.sum(wb_optimal_probs) for i in range(K)]
        sampling_probs = [(1 - gamma) * sampling_probs[i] + gamma * 1/K for i in range(K)]
# convert tracking_history to dataframe
df_WBTS = pd.DataFrame(tracking_history, columns=[
    'parallel_idx','null_vs_alter','batch_idx', 'arm_idx', 'sampling_prob', 
    'mu_nb', 'tau_nb', 'sample_variance_nb', 'optimal_prob_nb','posterior_delta_nb',
    'mu_wb', 'tau_wb', 'sample_variance_wb', 'optimal_prob_wb','posterior_delta_wb',
    'proportion_trials'
])
df_WBTS.to_csv(f"{saved_folder}/WBTS.csv", index=False)
del df_WBTS

# NB-TTTS

In [9]:
G = 5000
beta = 0.5
tracking_history = []
for parallel_idx in range(n_parallels):
    bandit_choice = parallel_idx % n_bandits
    null_vs_alter = bandit_choice
    if bandit_choice >= n_bandits//2:
        bandit = lst_bandit_null[bandit_choice - n_bandits//2]
    elif bandit_choice < n_bandits//2:
        bandit = lst_bandit_alter[bandit_choice]
    trials = [0 for i in range(K)]
    rewards = [0 for i in range(K)]
    mus_nb = [0 for i in range(K)]
    mus_wb = [0 for i in range(K)]
    lambdas = [0 for i in range(K)]
    numerator = [0 for i in range(K)]
    denominator = [0 for i in range(K)]
    batch_count = [0 for i in range(K)]
    # cal sample variance
    SS = [0 for _ in range(K)]
    sample_variance_nb = [1 for _ in range(K)]
    sample_variance_wb = [1 for _ in range(K)]
    sampling_probs = [1 / K for i in range(K)]
    for batch_idx in range(n_batches):
        batch_feedback = bandit.batch_observe_fast(
            sampling_distrib = sampling_probs, 
            time_index=batch_idx
        )
        for arm_idx in range(K):
            batch_trials = batch_feedback['trials'][arm_idx]
            if batch_trials > 0:
                trials[arm_idx] += batch_trials
                rewards[arm_idx] += batch_feedback['rewards'][arm_idx]
                # update batch info
                batch_count[arm_idx] += 1
                denominator[arm_idx] += np.sqrt(batch_trials)
                numerator[arm_idx] += batch_feedback['rewards'][arm_idx] / np.sqrt(batch_trials)
                # initial prior by first batch or update posterior
                lambdas[arm_idx] =  denominator[arm_idx]**2 / batch_count[arm_idx]
                mus_wb[arm_idx] = numerator[arm_idx] / denominator[arm_idx]
                mus_nb[arm_idx] = rewards[arm_idx] / trials[arm_idx]
                # cal sample variance
                SS[arm_idx] += batch_feedback['squared_rewards'][arm_idx]
                sample_variance_nb[arm_idx] = SS[arm_idx] / trials[arm_idx] - mus_nb[arm_idx]**2
                sample_variance_wb[arm_idx] = SS[arm_idx] / trials[arm_idx] - mus_wb[arm_idx]**2
        # weighted batched
        mc_sampling = np.array([np.random.normal(mus_wb[i], np.sqrt(sample_variance_wb[i]/(eta * lambdas[i])), G) for i in range(K)])
        mc_optimal_arm = np.argmax(mc_sampling, axis=0)
        wb_posterior_delta = np.mean(np.max(mc_sampling, axis=0) - mc_sampling, axis=1)
        wb_optimal_probs = [round(np.sum(np.where(mc_optimal_arm == arm_i, 1, 0)) / G, 4) for arm_i in range(K)]
        # naive batched
        mc_sampling = np.array([np.random.normal(mus_nb[i], np.sqrt(sample_variance_nb[i]/(eta * trials[i])), G) for i in range(K)])
        mc_optimal_arm = np.argmax(mc_sampling, axis=0)
        nb_posterior_delta = np.mean(np.max(mc_sampling, axis=0) - mc_sampling, axis=1)
        nb_optimal_probs = [round(np.sum(np.where(mc_optimal_arm == arm_i, 1, 0)) / G, 4) for arm_i in range(K)]
        # tracking history and analysis
        for arm_idx in range(K):
            tracking_reccord = (
                parallel_idx,
                null_vs_alter,
                batch_idx,
                arm_idx,
                sampling_probs[arm_idx],
                mus_nb[arm_idx],
                trials[arm_idx],
                sample_variance_nb[arm_idx],
                nb_optimal_probs[arm_idx],
                nb_posterior_delta[arm_idx],
                mus_wb[arm_idx],
                lambdas[arm_idx],
                sample_variance_wb[arm_idx],
                wb_optimal_probs[arm_idx],
                wb_posterior_delta[arm_idx],
                trials[arm_idx] / sum(trials)
            )
            tracking_history.append(tracking_reccord)
        # change traffic
        optimal_odds = [nb_optimal_probs[i] / (1 - nb_optimal_probs[i]) if nb_optimal_probs[i] < 1 else 99 for i in range(K)]
        sampling_probs = [nb_optimal_probs[i] * (beta + (1 - beta) * (np.sum(optimal_odds) - optimal_odds[i])) for i in range(K)]
        sampling_probs = [sampling_probs[i] / np.sum(sampling_probs) for i in range(K)]
        sampling_probs = [(1 - gamma) * sampling_probs[i] + gamma * 1/K for i in range(K)]
# convert tracking_history to dataframe
df_NBTTTS = pd.DataFrame(tracking_history, columns=[
    'parallel_idx','null_vs_alter','batch_idx', 'arm_idx', 'sampling_prob', 
    'mu_nb', 'tau_nb', 'sample_variance_nb', 'optimal_prob_nb','posterior_delta_nb',
    'mu_wb', 'tau_wb', 'sample_variance_wb', 'optimal_prob_wb','posterior_delta_wb',
    'proportion_trials'
])
df_NBTTTS.to_csv(f"{saved_folder}/NBTTTS.csv", index=False)
del df_NBTTTS

# WB-TTTS

In [10]:
G = 5000
beta = 0.5
tracking_history = []
for parallel_idx in range(n_parallels):
    bandit_choice = parallel_idx % n_bandits
    null_vs_alter = bandit_choice
    if bandit_choice >= n_bandits//2:
        bandit = lst_bandit_null[bandit_choice - n_bandits//2]
    elif bandit_choice < n_bandits//2:
        bandit = lst_bandit_alter[bandit_choice]
    trials = [0 for i in range(K)]
    rewards = [0 for i in range(K)]
    mus_nb = [0 for i in range(K)]
    mus_wb = [0 for i in range(K)]
    lambdas = [0 for i in range(K)]
    numerator = [0 for i in range(K)]
    denominator = [0 for i in range(K)]
    batch_count = [0 for i in range(K)]
    # cal sample variance
    SS = [0 for _ in range(K)]
    sample_variance_nb = [1 for _ in range(K)]
    sample_variance_wb = [1 for _ in range(K)]
    sampling_probs = [1 / K for i in range(K)]
    for batch_idx in range(n_batches):
        batch_feedback = bandit.batch_observe_fast(
            sampling_distrib = sampling_probs, 
            time_index=batch_idx
        )
        for arm_idx in range(K):
            batch_trials = batch_feedback['trials'][arm_idx]
            if batch_trials > 0:
                trials[arm_idx] += batch_trials
                rewards[arm_idx] += batch_feedback['rewards'][arm_idx]
                # update batch info
                batch_count[arm_idx] += 1
                denominator[arm_idx] += np.sqrt(batch_trials)
                numerator[arm_idx] += batch_feedback['rewards'][arm_idx] / np.sqrt(batch_trials)
                # initial prior by first batch or update posterior
                lambdas[arm_idx] =  denominator[arm_idx]**2 / batch_count[arm_idx]
                mus_wb[arm_idx] = numerator[arm_idx] / denominator[arm_idx]
                mus_nb[arm_idx] = rewards[arm_idx] / trials[arm_idx]
                # cal sample variance
                SS[arm_idx] += batch_feedback['squared_rewards'][arm_idx]
                sample_variance_nb[arm_idx] = SS[arm_idx] / trials[arm_idx] - mus_nb[arm_idx]**2
                sample_variance_wb[arm_idx] = SS[arm_idx] / trials[arm_idx] - mus_wb[arm_idx]**2
        # weighted batched
        mc_sampling = np.array([np.random.normal(mus_wb[i], np.sqrt(sample_variance_wb[i]/(eta * lambdas[i])), G) for i in range(K)])
        mc_optimal_arm = np.argmax(mc_sampling, axis=0)
        wb_posterior_delta = np.mean(np.max(mc_sampling, axis=0) - mc_sampling, axis=1)
        wb_optimal_probs = [round(np.sum(np.where(mc_optimal_arm == arm_i, 1, 0)) / G, 4) for arm_i in range(K)]
        # naive batched
        mc_sampling = np.array([np.random.normal(mus_nb[i], np.sqrt(sample_variance_nb[i]/(eta * trials[i])), G) for i in range(K)])
        mc_optimal_arm = np.argmax(mc_sampling, axis=0)
        nb_posterior_delta = np.mean(np.max(mc_sampling, axis=0) - mc_sampling, axis=1)
        nb_optimal_probs = [round(np.sum(np.where(mc_optimal_arm == arm_i, 1, 0)) / G, 4) for arm_i in range(K)]
        # tracking history and analysis
        for arm_idx in range(K):
            tracking_reccord = (
                parallel_idx,
                null_vs_alter,
                batch_idx,
                arm_idx,
                sampling_probs[arm_idx],
                mus_nb[arm_idx],
                trials[arm_idx],
                sample_variance_nb[arm_idx],
                nb_optimal_probs[arm_idx],
                nb_posterior_delta[arm_idx],
                mus_wb[arm_idx],
                lambdas[arm_idx],
                sample_variance_wb[arm_idx],
                wb_optimal_probs[arm_idx],
                wb_posterior_delta[arm_idx],
                trials[arm_idx] / sum(trials)
            )
            tracking_history.append(tracking_reccord)
        # change traffic
        optimal_odds = [wb_optimal_probs[i] / (1 - wb_optimal_probs[i]) if wb_optimal_probs[i] < 1 else 99 for i in range(K)]
        sampling_probs = [wb_optimal_probs[i] * (beta + (1 - beta) * (np.sum(optimal_odds) - optimal_odds[i])) for i in range(K)]
        sampling_probs = [sampling_probs[i] / np.sum(sampling_probs) for i in range(K)]
        sampling_probs = [(1 - gamma) * sampling_probs[i] + gamma * 1/K for i in range(K)]
# convert tracking_history to dataframe
df_WBTTTS = pd.DataFrame(tracking_history, columns=[
    'parallel_idx','null_vs_alter','batch_idx', 'arm_idx', 'sampling_prob', 
    'mu_nb', 'tau_nb', 'sample_variance_nb', 'optimal_prob_nb','posterior_delta_nb',
    'mu_wb', 'tau_wb', 'sample_variance_wb', 'optimal_prob_wb','posterior_delta_wb',
    'proportion_trials'
])
df_WBTTTS.to_csv(f"{saved_folder}/WBTTTS.csv", index=False)
del df_WBTTTS